In [1]:
from utils import *
st = time.time()

In [2]:
path="FB_100030____FB,2817673489/study_0c1e55cc/"
path_subfolder1=path+"MR7_bb3b1a0b"#"MR3_33a7c6a2"#"MR4_c61744e9"#"MR3_33a7c6a2"
path_subfolder2=path+"MR5_2e0c276a"
path_subfolder3=path+"MR3_33a7c6a2"

In [3]:
# read 2 series (sagittal / coronal / axial)
sag_image=serie_reader(path_subfolder1)
cor_image = serie_reader(path_subfolder2)
ax_image= serie_reader(path_subfolder3)

vType= sag_image.GetPixelIDTypeAsString()

In [4]:
# get spacing of each volume (dependent of orientation)
spacing_sag=get_spacing(sag_image)
spacing_cor=get_spacing(cor_image)
spacing_ax=get_spacing(ax_image)

print(spacing_sag, spacing_cor, spacing_ax)

[[0.4375    0.        0.       ]
 [0.        0.4375    0.       ]
 [0.        0.        3.3000002]] [[0.4375 0.     0.    ]
 [0.     0.4375 0.    ]
 [0.     0.     3.3   ]] [[0.4375 0.     0.    ]
 [0.     0.4375 0.    ]
 [0.     0.     3.6   ]]


In [5]:
# get direction cosin matrix of each volume (dependent of orientation)
dir_sag = get_direction(sag_image)
dir_cor = get_direction(cor_image)
dir_ax = get_direction(ax_image)

In [6]:
#dot product between spacing and Direct Cosine Matrix
A_sag=np.dot(dir_sag, spacing_sag)
A_cor=np.dot(dir_cor, spacing_cor)
A_ax=np.dot(dir_ax, spacing_ax)

In [7]:
#get origin from axial, sag, cor series
origin_sag= np.array(sag_image.GetOrigin())
origin_cor= np.array(cor_image.GetOrigin())
origin_ax = np.array(ax_image.GetOrigin())
print(origin_cor, origin_ax, origin_sag)

[  20.95518583 -102.72428033   48.84761293] [  29.43934254 -105.36338197  -81.23900868] [ 117.90852282 -103.56080763   53.28039366]


In [8]:
#get intensities from each series
m_sag=sitk.GetArrayFromImage(sag_image)
m_cor=sitk.GetArrayFromImage(cor_image)
m_ax=sitk.GetArrayFromImage(ax_image)

print(sag_image.GetSize(), m_sag.shape)

(320, 320, 32) (32, 320, 320)


In [9]:
# calcul physical coordinates of each volume
print("sag \n")
xspa_sag=calcul_physicsCoo(origin_sag, A_sag, m_sag)
print("cor \n")
xspa_cor=calcul_physicsCoo(origin_cor, A_cor, m_cor)
print("ax \n")
xspa_ax=calcul_physicsCoo(origin_ax, A_ax, m_ax)

sag 



100%|███████████████████████████████████████████| 32/32 [00:06<00:00,  5.32it/s]


cor 



100%|███████████████████████████████████████████| 34/34 [00:06<00:00,  5.10it/s]


ax 



100%|███████████████████████████████████████████| 31/31 [00:05<00:00,  5.54it/s]


In [10]:
#concat all points from each volume
sag_d= pd.DataFrame(xspa_sag, columns=('x', 'y', 'z'))
cor_d= pd.DataFrame(xspa_cor, columns=('x', 'y', 'z'))
ax_d= pd.DataFrame(xspa_ax, columns=('x', 'y', 'z'))

datas = [sag_d, cor_d, ax_d]
datas_combined = pd.concat(datas)
print(datas_combined.head())

newres=np.min(np.asarray([sag_image.GetSpacing(), cor_image.GetSpacing(), ax_image.GetSpacing()]))
print(newres)

            x           y          z
0  117.908523 -103.560808  53.280394
1  117.969399 -103.127564  53.280394
2  118.030276 -102.694320  53.280394
3  118.091152 -102.261076  53.280394
4  118.152028 -101.827832  53.280394
0.4375


In [11]:
#determine min coordinates, max coordinates

min_xyz=np.array([datas_combined['x'].min(), datas_combined['y'].min(), datas_combined['z'].min()]) 
max_xyz=np.array([datas_combined['x'].max(), datas_combined['y'].max(), datas_combined['z'].max()])
print(min_xyz.dtype, max_xyz)
new_origin, new_spacing, dir_new = define_param(newres, min_xyz, max_xyz)

sizeX=ceil((datas_combined['x'].max()-datas_combined['x'].min())/newres)
sizeY=ceil((datas_combined['y'].max()-datas_combined['y'].min())/newres)
sizeZ=ceil((datas_combined['z'].max()-datas_combined['z'].min())/newres)
print(newres, new_origin, new_spacing, dir_new, sizeX, sizeY, sizeZ)

float64 [164.71161767  62.8907225   56.71900472]
0.4375 [-10.49432384  62.8907225  -90.56740399] [[0.4375 0.     0.    ]
 [0.     0.4375 0.    ]
 [0.     0.     0.4375]] [[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]] 401 398 337


In [12]:
print(sag_d.x.min(), sag_d.y.min(), sag_d.z.min(), origin_sag)
print(m_sag.shape[0])

16.654379740823018 -104.1749785453054 -89.44767178682321 [ 117.90852282 -103.56080763   53.28039366]
32


In [13]:
def get_int(IJK, floorIJK, serie, newres):
    shapeX, shapeY, shapeZ = serie.shape 
    d= newres*3
    i= floorIJK[...,2].astype(np.int32)
    j= floorIJK[...,1].astype(np.int32)
    k= floorIJK[...,0].astype(np.int32)
    #print(i.shape, i)

    di= IJK[..., 2]-i
    dj= IJK[..., 1]-j
    dk= IJK[..., 0]-k

    condition_i = (i >= 0) & (i < shapeX - 3)
    condition_j = (j >= 0) & (j < shapeY - 3)
    condition_k = (k >= 0) & (k < shapeZ - 3)
    
    condition_di = (di <= d)
    condition_dj = (dj <= d)
    condition_dk = (dk <= d)

    # Combine the conditions for all three dimensions using logical AND
    condition_all = condition_i & condition_j & condition_k & condition_di & condition_dj & condition_dk

    # Create the new array (bounds) with values 1 where the condition is True, and 0 where it is False
    bounds = np.where(condition_all, 1, 0)

    i=i[np.where(bounds==1)]
    j=j[np.where(bounds==1)]
    k=k[np.where(bounds==1)]

    di=di[np.where(bounds==1)]
    dj=dj[np.where(bounds==1)]
    dk=dk[np.where(bounds==1)]
    
    # Create an array of zeros with the same shape as bounds
    new_vol = np.zeros((bounds.shape))
    new_vol[:] =np.nan
    # Apply the operation where bounds is equal to 1


    new_vol[bounds == 1]=(((1-di)*(1-dj)*(1-dk)*serie[i, j, k])+
              ((1-di)*(1-dj)*(dk)*serie[i, j, k+1]) +
              ((1-di)*(dj)*(1-dk)*serie[i, j+1, k])+
              ((di)*(1-dj)*(1-dk)*serie[i+1, j, k])+
              ((di)*(1-dj)*(dk)*serie[i+1, j, k+1])+
              ((di)*(dj)*(1-dk)*serie[i+1, j+1, k])+
              ((1-di)*(dj)*(dk)*serie[i, j+1, k+1])+
              ((di)*(dj)*(dk)*serie[i+1, j+1, k+1]))
    return new_vol


In [14]:
s=160 #128 crop

I=np.arange(int((sizeX/2)-s),int((sizeX/2)+s))
J=np.arange(int((sizeY/2)-s),int((sizeY/2)+s))
K=np.arange(int((sizeZ/2)-s),int((sizeZ/2)+s))
#np.dot(A_new, np.array([i,j,k]))
#print(I)
A_new=np.dot(dir_new, new_spacing)

ii, jj, kk = np.meshgrid(I,J,K, indexing='ij')
I012 = np.zeros((ii.shape[0], jj.shape[1], kk.shape[2], 3))
I012[:,:,:,0] = ii
I012[:,:,:,1] = jj
I012[:,:,:,2] = kk
print(ii.shape, jj.shape, kk.shape)
print(sizeX, sizeY, sizeZ)

XYZ=new_origin + np.einsum('ijkl,lm->ijkm', I012, A_new)
print(XYZ.shape)

# Calculate XYZ - origin_sag first
XYZ_minus_origin_sag = XYZ - origin_sag.reshape(1, 3)
XYZ_minus_origin_cor = XYZ - origin_cor.reshape(1, 3)
XYZ_minus_origin_ax = XYZ - origin_ax.reshape(1, 3)
invA_sag=np.linalg.inv(A_sag)
invA_cor=np.linalg.inv(A_cor)
invA_ax=np.linalg.inv(A_ax)

IJK_sag=np.matmul(XYZ_minus_origin_sag, invA_sag.T)
IJK_cor=np.matmul(XYZ_minus_origin_cor, invA_cor.T)
IJK_ax =np.matmul(XYZ_minus_origin_ax, invA_ax.T)

floor_IJK_sag=np.floor(IJK_sag)
floor_IJK_cor=np.floor(IJK_cor)
floor_IJK_ax =np.floor(IJK_ax)

sag_i=get_int(IJK_sag, floor_IJK_sag, m_sag, newres)
cor_i=get_int(IJK_cor, floor_IJK_cor, m_cor, newres)
ax_i=get_int(IJK_ax, floor_IJK_ax, m_ax, newres)
combined_vol = np.nanmean([cor_i, ax_i], axis=0)
#ijk_int=np.nanmean(np.array([sag_int, cor_int, ax_int]), axis=0)

(320, 320, 320) (320, 320, 320) (320, 320, 320)
401 398 337
(320, 320, 320, 3)


Mean of empty slice


In [15]:
print(np.sum(np.isnan(combined_vol)))
print(np.sum(~np.isnan(combined_vol)))
combined_vol=np.nan_to_num(combined_vol, nan=0).astype(np.int32)

5430832
27337168


In [16]:
plt_vol= k3d.volume(combined_vol.astype(np.float32), alpha_coef=75,color_map=matplotlib_color_maps.Turbo)
plot = k3d.plot()
plot += plt_vol

plot.display()

Output()

In [17]:
itk_view = itk_view_from_simpleitk(combined_vol.astype(np.float32), (newres, newres, newres), dir_new, new_origin)
#itk_view = itk_view_from_simpleitk(m_cor, cor_image.GetSpacing() , dir_cor, origin_cor)
itkwidgets.view(itk_view)

(320, 320, 320)


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [18]:
# Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
filename='VOL00_01TEST.nii.gz'
nv=combined_vol.astype(np.uint16)
result_image = sitk.Image(list(nv.shape), sitk.sitkUInt16)
result_image = sitk.GetImageFromArray(nv)
result_image.SetSpacing((newres, newres, newres))
result_image.SetOrigin(new_origin)
result_image.SetDirection(tuple(dir_new.flatten()))
    
# write the image
sitk.WriteImage(result_image, filename)

In [19]:
et = time.time()
# get the execution time
elapsed_time = (et - st)/60
print('Execution time:', elapsed_time, 'minutes')

Execution time: 0.7003578027089437 minutes
